# Goal: Extract facility metric data using the Contxt Python SDK

Contxt organizes its data in a Customer to Facility to Metric hierarchy. To retrieve metric data for all facilities, the general operation flow is:
1. Instance customer client
1. Locate customer facilities
1. Locate customer metric definitions
1. Retrieve metrics for each customer facility.

This document demonstrates this flow using the [Contxt Python SDK](https://github.com/ndustrialio/contxt-sdk-python).

Note: this example is functional but not optimized. At a minimum, multiprocessing could be used to speed up the data request for a large number of facilities.

In [1]:
from contxt.cli.clients import Clients

from random import randint
from datetime import datetime, timezone

# How long ago should we look for data?

The metric data retrieval service allows bounding a time window for returned data with `effective_start_time` and `effective_end_time` parameters. These values must be reported as ISO 8601 strings matching the `"%Y-%m-%dT%H:%M:%SZ"` format.

In this document, we're going to retrieve all data between now and the date in the `earliest_date` variable we declare below.

In [2]:
earliest_date = '2022-01-01T00:00:00Z'

# Initialize SDK client

If you are in a newly installed environment, you will need to configure your environment's Contxt authentication secrets with the CLI command: `contxt auth login` before running this notebook any further.

In [3]:
clients = Clients(env="production", org_slug="lineage")

# Get organization facilities

The next cell retrieves all of the organization's facilities.

In [4]:
facilities = clients.facilities.get_facilities()
len(facilities)

576

We'll use a randomly selected facility for the rest of the notebook

In [5]:
facilities = [facilities[randint(0, len(facilities) - 1)]]

# Retrieve Customer Metrics

## Locate the facility type definition

Contxt's dynamic nature allows for many asset types, but also leads to each organization having a unique definition for their asset types. The below code cell scans the asset type definitions for the organization and locates the definition for the `Facility` asset type. Most critically, later code will depend on the `Facility` asset type definition's unique id.

In [6]:
asset_types = clients.assets.get_asset_types()
facility_type = [atype for atype in asset_types if atype.label == 'Facility'][0]
facility_type

AssetType(label='Facility', description='Physical Facility Locations', organization_id='02efa741-a96f-4124-a463-ae13a704b8fc', id='616eee50-5dd3-4009-b8a3-dedfd8a7d56d', is_global=True, global_asset_type_parent_id='5f310899-d8f9-4dac-ae82-cedb2048a8ef', parent_id=None, hierarchy_level=1, created_at=datetime.datetime(2018, 11, 15, 16, 34, 17, 41000, tzinfo=datetime.timezone.utc), updated_at=datetime.datetime(2018, 11, 15, 16, 34, 17, 41000, tzinfo=datetime.timezone.utc))

## Retrieve Metric Definitions

A call to the SDK client's `get_metrics` function with the facility_type id will retrieve the full list of metric definitions. The below code cell demonstrates this with a filter applied to drop an unwanted definition.

In [7]:
metric_definitions = [mdef for mdef in clients.assets.get_metrics(facility_type.id) if mdef.label != "Blended Rate"]
print(f"Number of definitions: {len(metric_definitions)}")

Number of definitions: 69


## Retrieving Metric Values

Below, we make a python dataclass from our selected metrics. This will allow a degree of data validation as we build the full dataset over relying on simple dictionaries of data.

In [8]:
from dataclasses import make_dataclass, field
from typing import Any


FacilityEntry = make_dataclass(
    'FacilityEntry',
   ["facility", "date"] + [(mdef.label, Any, field(default=None)) for mdef in metric_definitions])

Next, we iterate over each metric for each facility to build out the full data set. Much of the work in this next code cell is in reorganizing the data returned by Contxt into a structure that allows building a single unified table. In this case, that structure is a list of items where each item is all of a Facility's metrics for a given date.

As written, this code can take about ~10 seconds per facility.

In [9]:
all_facility_data = []

# For each facility
for idx, facility in enumerate(facilities):
    facility_data = {}
    
    # For each metric of each facility
    for metric_def in metric_definitions:
        
        # Try to retrieve metric data, and skip this metric if it fails
        try:
            metric_values = clients.ems.get_metric_values(
                facility.asset_id, metric_def.label, params={"effective_start_date": earliest_date})
        except Exception:
            # print(f"\t{metric_def.label} not found for {facility.name}")
            continue
            
        # Reformat metric data into table structure
        for metric_value in metric_values:
            if metric_value.effective_start_date not in facility_data:
                facility_data[metric_value.effective_start_date] = {metric_def.label: metric_value.value}
            else:
                facility_data[metric_value.effective_start_date][metric_def.label] = metric_value.value
    all_facility_data.extend([FacilityEntry(facility.name, date, **fe) for date, fe in facility_data.items()])
    # print(f"Retrieve data for {facility.name}")
print(f"Retrieved {len(all_facility_data) * len(metric_definitions)} data points")

Retrieved 690 data points


# Now you have your data!

At this point, the facility data has been pulled into the python environment and can be manipulated or handed off to other tools. For example, this list of `FacilityEntry` can be converted directly to a Pandas dataframe like so:

In [10]:
import pandas
df = pandas.DataFrame(all_facility_data)
df.describe()

,facility_daily_active_meters,facility_daily_co2_factor,facility_daily_co2_per_unit,facility_daily_co2_tons,facility_daily_cubic_footage,facility_daily_electricity_spend,facility_daily_electricity_usage,facility_daily_energy_spend_per_lbs,facility_daily_energy_spend_per_unit,facility_daily_inbound_volume,...,facility_monthly_energy_spend_per_unit,facility_monthly_inbound_volume,facility_monthly_kwh_per_cuft,facility_monthly_kwh_per_lbs,facility_monthly_kwh_per_unit,facility_monthly_max_cuft,facility_monthly_outbound_volume,facility_monthly_production_units,facility_monthly_rolling_year_cubic_footage,facility_monthly_rolling_year_elec_kbtu
count,10.0,1.000000e+01,6.000000,10.000000,10.0,10.000000,10.000000,6.000000,6.000000,6.000000,...,1.000000,1.0,1.000000,1.00000,1.00000,1.0,1.0,1.0000,1.0,1.000000e+00
mean,2920.0,9.095380e-01,20.981941,7.612672,1882288.0,806.079477,16739.645774,1.110846,1.110846,532481.166667,...,1.279741,3194887.0,0.088932,26.57606,26.57606,1882288.0,3103881.9,6298.7689,1882288.0,2.068806e+07
std,0.0,1.170278e-16,10.895795,1.832227,0.0,193.999404,4028.918575,0.576724,0.576724,201968.832663,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,2920.0,9.095380e-01,6.769241,2.576079,1882288.0,272.905715,5664.587538,0.358375,0.358375,190333.000000,...,1.279741,3194887.0,0.088932,26.57606,26.57606,1882288.0,3103881.9,6298.7689,1882288.0,2.068806e+07
25%,2920.0,9.095380e-01,15.414544,7.588617,1882288.0,803.305825,16686.751063,0.816216,0.816216,442642.750000,...,1.279741,3194887.0,0.088932,26.57606,26.57606,1882288.0,3103881.9,6298.7689,1882288.0,2.068806e+07
50%,2920.0,9.095380e-01,19.882779,8.217151,1882288.0,870.242971,18068.846952,1.052621,1.052621,612202.000000,...,1.279741,3194887.0,0.088932,26.57606,26.57606,1882288.0,3103881.9,6298.7689,1882288.0,2.068806e+07
75%,2920.0,9.095380e-01,25.011785,8.377974,1882288.0,886.842608,18422.482725,1.324461,1.324461,665225.500000,...,1.279741,3194887.0,0.088932,26.57606,26.57606,1882288.0,3103881.9,6298.7689,1882288.0,2.068806e+07
max,2920.0,9.095380e-01,38.710258,8.904950,1882288.0,943.178983,19581.259941,2.048972,2.048972,711127.000000,...,1.279741,3194887.0,0.088932,26.57606,26.57606,1882288.0,3103881.9,6298.7689,1882288.0,2.068806e+07
